In [1]:
# CELL 1: Import libraries
import pandas as pd
import numpy as np
import re
import os

print("🔍 Checking available files...")
for file in os.listdir('.'):
    if file.endswith('.xlsx'):
        print(f"📄 {file}")

🔍 Checking available files...
📄 29 Oct - Simulate Data - US Space Template.xlsx


In [2]:
# CELL 2: Load Excel file properly
try:
    # Use the exact path from your error message
    file_path = "/home/x-rbenlabhili/Data_Simulation/29 Oct - Simulate Data - US Space Template.xlsx"
    
    # Load all sheets
    facility_df = pd.read_excel(file_path, sheet_name='Facility Condition Index')
    projects_df = pd.read_excel(file_path, sheet_name='Projects')
    
    print("✅ SUCCESS! Files loaded")
    print(f"📊 Facilities: {len(facility_df)} records")
    print(f"🔧 Projects: {len(projects_df)} records")
    
    # Show first few rows to confirm
    print("\n📋 Facility data sample:")
    print(facility_df[['Site', 'Title', 'Facility Number']].head())
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 Trying alternative approach...")
    
    # Alternative: List all sheets to see what's available
    try:
        xl_file = pd.ExcelFile(file_path)
        print(f"Available sheets: {xl_file.sheet_names}")
        xl_file.close()
    except Exception as e2:
        print(f"Also failed: {e2}")

✅ SUCCESS! Files loaded
📊 Facilities: 226 records
🔧 Projects: 64 records

📋 Facility data sample:
     Site                      Title Facility Number
0  Site A    Heating - Cooling Plant            1783
1  Site A   Antenna support building            2318
2  Site A     Communication Facility            1039
3  Site A  Power generation Facility            4151
4  Site A         Ground Terminal T1            3550


In [5]:
# CELL 4: Calculate Condition Indices
def calculate_condition_indices(facility_df, projects_df):
    """Calculate realistic Condition Indices"""
    
    # If we have real project data, use it
    cost_to_ci = {}
    if len(projects_df) > 0:
        for _, project in projects_df.iterrows():
            if pd.notna(project.get('Cost')) and pd.notna(project.get('Scope')):
                cost_str = str(project['Cost'])
                scope = str(project['Scope'])
                
                try:
                    # Extract cost
                    if 'M' in cost_str:
                        cost_val = float(cost_str.replace('$', '').replace('M', '').strip())
                    else:
                        cost_val = 1.0
                    
                    # Find building number
                    building_match = re.search(r'Building (\d+)', scope)
                    if building_match:
                        bld_num = int(building_match.group(1))
                        
                        # Map cost to condition
                        if cost_val > 20:
                            cost_to_ci[bld_num] = np.random.randint(20, 35)
                        elif cost_val > 10:
                            cost_to_ci[bld_num] = np.random.randint(35, 50)
                        elif cost_val > 5:
                            cost_to_ci[bld_num] = np.random.randint(50, 65)
                        elif cost_val > 1:
                            cost_to_ci[bld_num] = np.random.randint(65, 80)
                        else:
                            cost_to_ci[bld_num] = np.random.randint(75, 90)
                except:
                    continue
    
    # Apply to facilities
    condition_indices = []
    data_sources = []
    
    for _, facility in facility_df.iterrows():
        facility_num = facility['Facility Number']
        
        if facility_num in cost_to_ci:
            ci = cost_to_ci[facility_num]
            source = 'Project Data'
        else:
            # Use distribution rules from your specs
            rand = np.random.random()
            if rand < 0.07:
                ci = np.random.randint(1, 50)    # 7% poor condition
            elif rand < 0.12:
                ci = np.random.randint(85, 100)  # 5% excellent condition
            else:
                ci = np.random.randint(50, 85)   # 88% fair to good
            source = 'Distribution Rules'
        
        condition_indices.append(ci)
        data_sources.append(source)
    
    return condition_indices, data_sources

# Calculate Condition Indices
condition_indices, data_sources = calculate_condition_indices(facility_df, projects_df)
facility_df['Condition Index'] = condition_indices
facility_df['CI Data Source'] = data_sources

print(" Condition Indices calculated!")

 Condition Indices calculated!


In [6]:
# CELL 5: Display results
print("CONDITION INDEX RESULTS")
print("-" * 70)

# Show results
display_cols = ['Unique Identifier', 'Site', 'Title', 'Facility Number', 'Condition Index', 'CI Data Source']
if 'Site' in facility_df.columns:
    display_df = facility_df[display_cols].head(20)
else:
    display_df = facility_df[['Unique Identifier', 'Title', 'Facility Number', 'Condition Index', 'CI Data Source']].head(20)

print(display_df.to_string(index=False))

print(f"\n📈 SUMMARY STATISTICS:")
print(f"Total facilities: {len(facility_df)}")
print(f"Average Condition Index: {facility_df['Condition Index'].mean():.1f}")
print(f"Data sources: {facility_df['CI Data Source'].value_counts().to_dict()}")

CONDITION INDEX RESULTS
----------------------------------------------------------------------
Unique Identifier   Site                     Title Facility Number  Condition Index     CI Data Source
                1 Site A   Heating - Cooling Plant            1783               73 Distribution Rules
                2 Site A  Antenna support building            2318               67 Distribution Rules
                3 Site A    Communication Facility            1039               84 Distribution Rules
                4 Site A Power generation Facility            4151               73 Distribution Rules
                5 Site A        Ground Terminal T1            3550               72 Distribution Rules
                6 Site A           Ops Facility T1            1225                1 Distribution Rules
                7 Site A               Data Center            2702               63 Distribution Rules
                8 Site A          Receiver Station             200               